## Project Name:

**Group 15: Alice Le, Jitao Zhang, Lincoln Lee, Yitong Gong**

In [7]:
# All needed libraries:
library(tidyverse)
library(broom)
library(repr)
library(digest)
library(infer)
library(gridExtra)

# General Graphs' setting:
options(repr.plot.width = 10, repr.plot.height = 6)

#### __Preliminary Results__

In [10]:
original_dataset <- read_csv("data/nces330_20.csv")
head(original_dataset)

Rows: 3548 Columns: 6
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (4): State, Type, Length, Expense
dbl (2): Year, Value

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Year,State,Type,Length,Expense,Value
<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>
2013,Alabama,Private,4-year,Fees/Tuition,13983
2013,Alabama,Private,4-year,Room/Board,8503
2013,Alabama,Public In-State,2-year,Fees/Tuition,4048
2013,Alabama,Public In-State,4-year,Fees/Tuition,8073
2013,Alabama,Public In-State,4-year,Room/Board,8473
2013,Alabama,Public Out-of-State,2-year,Fees/Tuition,7736


ERROR: Error in nrows(original_dataset): could not find function "nrows"


In [30]:
set.seed(2356)
tuition <- original_dataset |>
    filter(!is.na(Value),
           Type == "Public In-State",
           Length == "4-year",
           Expense == "Fees/Tuition",
           Year == 2020)

# tuition <- original_dataset |>
#     filter(Year == 2021,
#            Type == "Public In-State",
#            Length == "4-year",
#            Expense == "Fees/Tuition")
tuition

Year,State,Type,Length,Expense,Value
<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>
2013,Alabama,Public In-State,4-year,Fees/Tuition,8073
2013,Alaska,Public In-State,4-year,Fees/Tuition,6317
2013,Arizona,Public In-State,4-year,Fees/Tuition,9694
2013,Arkansas,Public In-State,4-year,Fees/Tuition,6604
2013,California,Public In-State,4-year,Fees/Tuition,8892
2013,Colorado,Public In-State,4-year,Fees/Tuition,7656
2013,Connecticut,Public In-State,4-year,Fees/Tuition,9517
2013,Delaware,Public In-State,4-year,Fees/Tuition,10929
2013,District of Columbia,Public In-State,4-year,Fees/Tuition,7244
